In [11]:
import os
import shutil
import numpy as np
import ast
import pandas as pd

In [2]:
def organize_files_by_number(folder_path):
    # Ensure the folder exists
    if not os.path.exists(folder_path):
        print(f"Error: The folder '{folder_path}' does not exist.")
        return

    # Create subfolders for numbers 128 to 137
    for number in range(128, 138):
        subfolder_path = os.path.join(folder_path, str(number))
        os.makedirs(subfolder_path, exist_ok=True)

    # Iterate over all .txt files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            # Extract the number after 'SNMOT-'
            parts = filename.split('-')
            if len(parts) > 1:
                number_part = parts[1].split('_')[0]
                if number_part.isdigit():
                    number = int(number_part)

                    # Move the file to the respective folder if the number is in range
                    if 128 <= number <= 137:
                        source_path = os.path.join(folder_path, filename)
                        destination_path = os.path.join(folder_path, str(number), filename)
                        shutil.move(source_path, destination_path)




In [3]:
# Example usage
# Provide the path to the 'labels' folder
#organize_files_by_number("C:\Studie\BEP\BEP_Football_tracking\Dataset_ball_eval\labels")

In [72]:
def process_text_files(input_folder):
    """
    Process all .txt files in the given folder. Transform the content by:
        - Removing the first number (always 0).
        - Multiplying the next four numbers by 1920, 1080, 1920, and 1080 respectively.
        - Modifying position 1 and 2 by subtracting half of position 3 and 4, respectively.
        - Rounding the results to the nearest integer.
        - Saving the result in the format [1, 2, 3, 4].

    Args:
        input_folder (str): Path to the folder containing the .txt files.
    """
    for filename in os.listdir(input_folder):
        if filename.endswith(".txt"):
            file_path = os.path.join(input_folder, filename)

            # Read the file content
            with open(file_path, 'r') as file:
                lines = file.readlines()

            # If the file is empty, skip processing
            if not lines:
                continue

            # Process the first line (ignoring additional lines if any)
            line = lines[0].strip().split()
            if len(line) >= 5:  # Ensure the line has enough numbers to process
                x1 = float(line[1]) * 1920
                y1 = float(line[2]) * 1080
                x2 = float(line[3]) * 1920
                y2 = float(line[4]) * 1080

                # Modify position 1 and 2
                x1 = round(x1 - (x2 / 2))
                y1 = round(y1 - (y2 / 2))

                x2 = round(x2)
                y2 = round(y2)

                # Format the transformed data
                transformed_line = f"[{x1}, {y1}, {x2}, {y2}]\n"

                # Write the transformed data back to the file
                with open(file_path, 'w') as file:
                    file.write(transformed_line)

In [73]:
# Example Usage:
#process_text_files('C:/Studie/BEP/BEP_Football_tracking/Dataset_ball_eval/labels/133')


In [4]:
def bb_intersection_over_union(boxA, boxB):
    # Convert from [xtopleft, ytopleft, width, height] to [x1, y1, x2, y2]
    xA1, yA1, wA, hA = boxA
    xB1, yB1, wB, hB = boxB
    xA2, yA2 = xA1 + wA, yA1 + hA
    xB2, yB2 = xB1 + wB, yB1 + hB

    # Determine the (x, y)-coordinates of the intersection rectangle
    x_intersect1 = max(xA1, xB1)
    y_intersect1 = max(yA1, yB1)
    x_intersect2 = min(xA2, xB2)
    y_intersect2 = min(yA2, yB2)

    # Compute the area of the intersection rectangle
    interArea = max(0, x_intersect2 - x_intersect1) * max(0, y_intersect2 - y_intersect1)

    # Compute the area of both the prediction and ground-truth rectangles
    boxAArea = wA * hA
    boxBArea = wB * hB

    # Compute the intersection over union by taking the intersection area
    # and dividing it by the sum of prediction + ground-truth areas - intersection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    return iou

In [3]:
def process_lines_to_lists(folder_path):
    """
    Reads each .txt file in the given folder, converts each non-empty line to a Python list,
    and stores all lists (or 0 for empty lines or empty files) in a main list.

    Args:
        folder_path (str): The path to the folder containing the .txt files.

    Returns:
        list: A list containing lists from each non-empty line or 0 for empty lines or empty files.
    """
    all_lists = []

    # List and sort all files in the folder by name
    sorted_files = sorted([f for f in os.listdir(folder_path) if f.endswith(".txt")])

    for filename in sorted_files:
        # Construct the full path of the file
        file_path = os.path.join(folder_path, filename)

        # Open the file and read each line
        with open(file_path, 'r') as file:
            lines = file.readlines()

            # Check if the file is empty
            if not lines:
                all_lists.append(0)  # Store 0 for empty files
            else:
                for line in lines:
                    # Strip leading/trailing whitespace
                    line = line.strip()

                    if line:
                        try:
                            # Convert the string to a Python list using eval
                            python_list = eval(line)
                            if isinstance(python_list, list):
                                all_lists.append(python_list)
                            else:
                                print(f"Line in {filename} is not a valid list: {line}")
                                all_lists.append(0)  # Store 0 if it's not a valid list
                        except (SyntaxError, NameError) as e:
                            print(f"Error parsing line in {filename}: {line}")
                            all_lists.append(0)  # Store 0 if there's an error
                    else:
                        all_lists.append(0)  # Store 0 for empty lines

    return all_lists

In [10]:
def evaluate_lists(groundtruth, prediction):
    """
    Evaluates two lists, `groundtruth` and `prediction`, and calculates metrics based on specific conditions.

    Parameters:
    groundtruth (list): The ground truth list.
    prediction (list): The prediction list.
    bb_intersection_over_union (function): A function to calculate IOU for two bounding boxes.

    Returns:
    dict: A dictionary containing TN, FP, FN, and IOU values for matched bounding boxes.
    """
    # Initialize counters
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    iou_thresh = 0.1

    # Iterate over both lists
    for gt, pred in zip(groundtruth, prediction):
        if isinstance(gt, int) and gt == 0:
            if isinstance(pred, int) and pred == 0:
                TN += 1  # True Negative
            elif isinstance(pred, list):
                FP += 1  # False Positive
        elif isinstance(gt, list):
            if isinstance(pred, int) and pred == 0:
                FN += 1  # False Negative
            elif isinstance(pred, list):
                # Both are lists, calculate IOU
                iou = bb_intersection_over_union(gt, pred)

                if iou >= iou_thresh:
                    TP += 1
                else:
                    FP += 1

    # Return the results
    return {
        'TP': TP,
        'TN': TN,
        'FP': FP,
        'FN': FN
    }



In [11]:
gt = process_lines_to_lists('C:/Studie/BEP/BEP_Football_tracking/Dataset_ball_eval/labelsfolders/137')
pred = process_lines_to_lists('C:/Studie/BEP/BEP_Football_tracking/Dataset_ball_eval/sequences/labels/137')

results = evaluate_lists(gt, pred)
results

{'TP': 604, 'TN': 0, 'FP': 74, 'FN': 72}

In [11]:
def rename_files_with_leading_zeroes(folder_path):
    try:
        # List all files in the folder
        files = os.listdir(folder_path)

        for file_name in files:
            # Ensure we only process .txt files
            if file_name.endswith('.txt'):
                # Extract the file name without extension
                base_name = os.path.splitext(file_name)[0]

                # Ensure the base_name is numeric to avoid errors
                if base_name.isdigit():
                    # Format the name with leading zeroes to make it 5 digits
                    new_name = f"{int(base_name):05}.txt"

                    # Generate full paths
                    old_file_path = os.path.join(folder_path, file_name)
                    new_file_path = os.path.join(folder_path, new_name)

                    # Rename the file
                    os.rename(old_file_path, new_file_path)
                    print(f"Renamed: {file_name} -> {new_name}")

    except Exception as e:
        print(f"An error occurred: {e}")



In [21]:
rename_files_with_leading_zeroes('C:/Studie/BEP/BEP_Football_tracking/Dataset_ball_eval/onlysamurai/137')

Renamed: 1.txt -> 00001.txt
Renamed: 10.txt -> 00010.txt
Renamed: 100.txt -> 00100.txt
Renamed: 101.txt -> 00101.txt
Renamed: 102.txt -> 00102.txt
Renamed: 103.txt -> 00103.txt
Renamed: 104.txt -> 00104.txt
Renamed: 105.txt -> 00105.txt
Renamed: 106.txt -> 00106.txt
Renamed: 107.txt -> 00107.txt
Renamed: 108.txt -> 00108.txt
Renamed: 109.txt -> 00109.txt
Renamed: 11.txt -> 00011.txt
Renamed: 110.txt -> 00110.txt
Renamed: 111.txt -> 00111.txt
Renamed: 112.txt -> 00112.txt
Renamed: 113.txt -> 00113.txt
Renamed: 114.txt -> 00114.txt
Renamed: 115.txt -> 00115.txt
Renamed: 116.txt -> 00116.txt
Renamed: 117.txt -> 00117.txt
Renamed: 118.txt -> 00118.txt
Renamed: 119.txt -> 00119.txt
Renamed: 12.txt -> 00012.txt
Renamed: 120.txt -> 00120.txt
Renamed: 121.txt -> 00121.txt
Renamed: 122.txt -> 00122.txt
Renamed: 123.txt -> 00123.txt
Renamed: 124.txt -> 00124.txt
Renamed: 125.txt -> 00125.txt
Renamed: 126.txt -> 00126.txt
Renamed: 127.txt -> 00127.txt
Renamed: 128.txt -> 00128.txt
Renamed: 129.tx

In [69]:
import os

def filter_txt_files(folder_path):
    """
    Removes all lines in .txt files within the specified folder
    that do not start with "0.0".

    :param folder_path: Path to the folder containing .txt files.
    """
    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"The folder {folder_path} does not exist.")

    # Iterate over all files in the folder
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)

        # Process only .txt files
        if os.path.isfile(file_path) and file_name.endswith('.txt'):
            with open(file_path, 'r') as file:
                lines = file.readlines()

            # Filter lines that start with "0.0"
            filtered_lines = [line for line in lines if line.startswith('0.0')]

            # Write the filtered lines back to the file
            with open(file_path, 'w') as file:
                file.writelines(filtered_lines)


In [4]:
import os

def filter_txt_files(folder_path):
    """
    Removes all lines in .txt files within the specified folder
    that do not start with "0.0".

    :param folder_path: Path to the folder containing .txt files.
    """
    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"The folder {folder_path} does not exist.")

    # Iterate over all files in the folder
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)

        # Process only .txt files
        if os.path.isfile(file_path) and file_name.endswith('.txt'):
            with open(file_path, 'r') as file:
                lines = file.readlines()

            # Filter lines that start with "0.0"
            filtered_lines = [line for line in lines if line.startswith('0.0')]

            # Write the filtered lines back to the file
            with open(file_path, 'w') as file:
                file.writelines(filtered_lines)



In [5]:
# Example usage:
filter_txt_files("C:/Studie/BEP/BEP_Football_tracking/Dataset_ball_eval/temp")

In [14]:
data = {
    "Sequence": [128, 129, 130, 131, 132, 133, 134, 135, 136, 137],
    "TP": [464, 445, 287, 383, 540, 556, 511, 475, 434, 604],
    "FP": [77, 86, 262, 73, 167, 150, 196, 53, 198, 74],
    "TN": [98, 181, 112, 180, 1, 1, 1, 3, 5, 0],
    "FN": [111, 38, 89, 114, 43, 43, 43, 219, 113, 72]
}

# Convert data into a pandas DataFrame
df = pd.DataFrame(data)

# Compute Precision, Recall, F1-Score, and Accuracy
def compute_metrics(row):
    tp = row['TP']
    fp = row['FP']
    tn = row['TN']
    fn = row['FN']

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0

    return precision, recall, f1_score, accuracy

# Apply metrics calculation to each row
df[['Precision', 'Recall', 'F1-Score', 'Accuracy']] = df.apply(
    lambda row: pd.Series(compute_metrics(row)), axis=1
)

# Display the DataFrame
print(df)

# Compute totals for all sequences
total_tp = df['TP'].sum()
total_fp = df['FP'].sum()
total_tn = df['TN'].sum()
total_fn = df['FN'].sum()

overall_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
overall_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
overall_f1_score = (2 * overall_precision * overall_recall) / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0
overall_accuracy = (total_tp + total_tn) / (total_tp + total_tn + total_fp + total_fn) if (total_tp + total_tn + total_fp + total_fn) > 0 else 0

print("\nOverall Metrics:")
print(f"Precision: {overall_precision:.4f}")
print(f"Recall: {overall_recall:.4f}")
print(f"F1-Score: {overall_f1_score:.4f}")
print(f"Accuracy: {overall_accuracy:.4f}")

   Sequence   TP   FP   TN   FN  Precision    Recall  F1-Score  Accuracy
0       128  464   77   98  111   0.857671  0.806957  0.831541  0.749333
1       129  445   86  181   38   0.838041  0.921325  0.877712  0.834667
2       130  287  262  112   89   0.522769  0.763298  0.620541  0.532000
3       131  383   73  180  114   0.839912  0.770624  0.803778  0.750667
4       132  540  167    1   43   0.763791  0.926244  0.837209  0.720373
5       133  556  150    1   43   0.787535  0.928214  0.852107  0.742667
6       134  511  196    1   43   0.722772  0.922383  0.810468  0.681758
7       135  475   53    3  219   0.899621  0.684438  0.777414  0.637333
8       136  434  198    5  113   0.686709  0.793419  0.736217  0.585333
9       137  604   74    0   72   0.890855  0.893491  0.892171  0.805333

Overall Metrics:
Precision: 0.7786
Recall: 0.8415
F1-Score: 0.8088
Accuracy: 0.7039


In [15]:
import pandas as pd

data = {
    "Sequence": [128, 129, 130, 131, 132, 133, 134, 135, 136, 137],
    "TP": [298, 5, 293, 297, 423, 603, 494, 256, 152, 531],
    "FP": [452, 745, 457, 453, 327, 147, 256, 494, 598, 219],
    "TN": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "FN": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
}

# Convert data into a pandas DataFrame
df = pd.DataFrame(data)

# Compute Precision, Recall, F1-Score, and Accuracy
def compute_metrics(row):
    tp = row['TP']
    fp = row['FP']
    tn = row['TN']
    fn = row['FN']

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0

    return precision, recall, f1_score, accuracy

# Apply metrics calculation to each row
df[['Precision', 'Recall', 'F1-Score', 'Accuracy']] = df.apply(
    lambda row: pd.Series(compute_metrics(row)), axis=1
)

# Display the DataFrame
print(df)

# Compute totals for all sequences
total_tp = df['TP'].sum()
total_fp = df['FP'].sum()
total_tn = df['TN'].sum()
total_fn = df['FN'].sum()

overall_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
overall_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
overall_f1_score = (2 * overall_precision * overall_recall) / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0
overall_accuracy = (total_tp + total_tn) / (total_tp + total_tn + total_fp + total_fn) if (total_tp + total_tn + total_fp + total_fn) > 0 else 0

print("\nOverall Metrics:")
print(f"Precision: {overall_precision:.4f}")
print(f"Recall: {overall_recall:.4f}")
print(f"F1-Score: {overall_f1_score:.4f}")
print(f"Accuracy: {overall_accuracy:.4f}")

   Sequence   TP   FP  TN  FN  Precision  Recall  F1-Score  Accuracy
0       128  298  452   0   0   0.397333     1.0  0.568702  0.397333
1       129    5  745   0   0   0.006667     1.0  0.013245  0.006667
2       130  293  457   0   0   0.390667     1.0  0.561841  0.390667
3       131  297  453   0   0   0.396000     1.0  0.567335  0.396000
4       132  423  327   0   0   0.564000     1.0  0.721228  0.564000
5       133  603  147   0   0   0.804000     1.0  0.891353  0.804000
6       134  494  256   0   0   0.658667     1.0  0.794212  0.658667
7       135  256  494   0   0   0.341333     1.0  0.508946  0.341333
8       136  152  598   0   0   0.202667     1.0  0.337029  0.202667
9       137  531  219   0   0   0.708000     1.0  0.829040  0.708000

Overall Metrics:
Precision: 0.4469
Recall: 1.0000
F1-Score: 0.6178
Accuracy: 0.4469


In [8]:
# Data from the table
confidence_data = [
    [0.001, 3765, 412, 578, 2745],
    [0.10, 3928, 265, 455, 2582],
    [0.20, 3826, 168, 822, 2684],
    [0.30, 3622, 99, 891, 2888],
    [0.40, 3215, 44, 946, 3295],
    [0.50, 2670, 24, 966, 3840],
    [0.60, 2132, 11, 979, 4378],
    [0.70, 1675, 4, 986, 4835],
    [0.80, 1142, 1, 989, 5368],
    [0.90, 0, 0, 990, 6510],
]

# Define column names
columns = ["Confidence Threshold", "TP", "FP", "TN", "FN"]

# Create DataFrame
data = pd.DataFrame(confidence_data, columns=columns)

def compute_metrics(tp, fp, tn, fn, beta=0.5):
    """
    Compute precision, recall, F1-score, accuracy, and F-beta score.
    """
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    accuracy = (tp + tn) / (tp + fp + tn + fn) if (tp + fp + tn + fn) > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    fbeta_score = (
        (1 + beta**2) * precision * recall / (beta**2 * precision + recall)
        if (beta**2 * precision + recall) > 0
        else 0
    )
    return precision, recall, f1_score, accuracy, fbeta_score

# Add metrics to the DataFrame
data["Precision"] = 0
data["Recall"] = 0
data["F1-Score"] = 0
data["Accuracy"] = 0
data["F-beta (beta=2)"] = 0

for i, row in data.iterrows():
    tp, fp, tn, fn = row["TP"], row["FP"], row["TN"], row["FN"]
    precision, recall, f1_score, accuracy, fbeta_score = compute_metrics(tp, fp, tn, fn, beta=0.5)
    data.at[i, "Precision"] = precision
    data.at[i, "Recall"] = recall
    data.at[i, "F1-Score"] = f1_score
    data.at[i, "Accuracy"] = accuracy
    data.at[i, "F-beta (beta=2)"] = fbeta_score

# Print the DataFrame
print(data)


   Confidence Threshold    TP   FP   TN    FN  Precision    Recall  F1-Score  \
0                 0.001  3765  412  578  2745   0.901365  0.578341  0.704594   
1                 0.100  3928  265  455  2582   0.936799  0.603379  0.734000   
2                 0.200  3826  168  822  2684   0.957937  0.587711  0.728484   
3                 0.300  3622   99  891  2888   0.973394  0.556375  0.708044   
4                 0.400  3215   44  946  3295   0.986499  0.493856  0.658205   
5                 0.500  2670   24  966  3840   0.991091  0.410138  0.580183   
6                 0.600  2132   11  979  4378   0.994867  0.327496  0.492777   
7                 0.700  1675    4  986  4835   0.997618  0.257296  0.409085   
8                 0.800  1142    1  989  5368   0.999125  0.175422  0.298445   
9                 0.900     0    0  990  6510   0.000000  0.000000  0.000000   

   Accuracy  F-beta (beta=2)  
0  0.579067         0.810793  
1  0.606224         0.843570  
2  0.619733         0.8507

C:\Users\20212189\AppData\Local\Temp\ipykernel_17684\4292000954.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.9013646157529327' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.at[i, "Precision"] = precision
C:\Users\20212189\AppData\Local\Temp\ipykernel_17684\4292000954.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5783410138248848' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.at[i, "Recall"] = recall
C:\Users\20212189\AppData\Local\Temp\ipykernel_17684\4292000954.py:48: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.7045943669879292' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.at

NameError: name 'go' is not defined